In [1]:

"""
C(64)–C(32)-BL(8)-BL(16)-L(32)-L(32)
with dropout only. 

Results of this code.
Train Score: 0.22578 RMSE
Test Score: 0.82191 RMSE
--------------------------------------------------------
 history.losses      =  0.0010714132143226879
 history.mses(=loss) =  0.0010714132143226879
 history.maes        =  0.024538412532253175
 history.mapes       =  1862.5542151558561

 history.val_losses     =  0.01422208130222434
 history.val_mses(=loss)=  0.01422208130222434
 history.val_maes       =  0.11461550989645886
 history.val_mapes      =  1869.1437989478238

"""


# Version. 
# 12.15. conv.2layers 
# 12.16. conv.4layers  
# 
# 유의사항: 절대하지말것. conv1d에서는 BN, Pooling적용 시, 언더피팅, 퍼포먼스 감소. 
# 삭제 visible1 = Input(shape=(204770, 29))  #### 아래 strides를 1로 해야 acc향상 
# 추가 visible1 = Input(shape=(timesteps, features))  # 29, 1
# 성공 
# cnn에서 stride를 1로 해야 accuracy 향상, 속도 느려짐. 
"""  
2018.12.11.화.23시10분. success  
2018.12.12.수.0시. sucess. shortcut(skip) connection, residual net. 

"""
#  Timestep = 29,  (19는 Loss크다)
#  A Type code : User defined History Class 
#       class LossHistory(keras.callbacks.Callback):  #history = LossHistory()
#             def ... def ... 
#        model.fit( )  # # because of if loop for fit() 
#  B Type code : Default typed History Class 
#       compile 
#       history = model.fit( )
#       score = model.evaluate 
# Kwangsuk.Lee, 2018.6.12.
import numpy
import matplotlib.pyplot as plt
import pandas     
from pandas import read_csv
from pandas import DataFrame 
from pandas import Series 
import math
import keras      
from keras.models import Sequential, Model 
from keras.layers import Dense, Input, Flatten, Add, concatenate 
from keras.layers import Activation, BatchNormalization, regularizers, Dropout

#from keras.layers.noise import GaussianNoise
#from keras.layers import Conv1D, MaxPooling1D  
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# keras.layers.pooling.GlobalMaxPooling1D() no input parameters.
from keras.layers.merge import concatenate

from keras.layers import LSTM, Bidirectional 
from keras.utils import plot_model  # New 

from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.regularizers import l2

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error #Mean squared error regression loss
from sklearn.metrics import mean_absolute_error # Mean absolute error regression loss
# http://scikit-learn.org/stable/modules/classes.html 
#http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# http://scikit-learn.org/stable/modules/classes.html
from sklearn.metrics import recall_score, precision_score # New 
from sklearn.metrics import f1_score #f1_score(y_true, y_pred[, labels, …])
from sklearn.metrics import confusion_matrix 
from sklearn import preprocessing  # New 
#from keras.layers import normalization ### 추가 keras.layers.normalization.BatchNormalization() 
#from keras.layers import Dropout  ### 추가 keras.layers.Dropout()


title = '190320_07_dropout_0.5_' 
epochs = 10 
batch_size = 128 # 64, 128, do not 256, i.e. in case of 128, 1 epoch = 9minutes     

class LossHistory(keras.callbacks.Callback):   #history = LossHistory()
    def init(self):         # history.init() 
        self.losses = []
        #self.accs = []
        self.val_losses = []
        #self.val_accs = []
        self.mses = []
        self.maes = []
        self.mapes = [] 
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []   
    
    def on_epoch_end(self, batch, logs={}): 
        self.losses.append(logs.get('loss'))
        #self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        #self.val_accs.append(logs.get('val_accuracy')) 
        self.mses.append(logs.get('mean_squared_error')) 
        self.maes.append(logs.get('mean_absolute_error')) 
        self.mapes.append(logs.get('mean_absolute_percentage_error')) 
        self.val_mses.append(logs.get('val_mean_squared_error')) 
        self.val_maes.append(logs.get('val_mean_absolute_error')) 
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))       
# end of the class 

# convert an array of values into a dataset matrix
def create_dataset(dataset, timesteps): #기존def create_dataset(dataset, timesteps=1): 여기서 -1삭제
    dataX, dataY = [], []
    for i in range(len(dataset)-timesteps-1):
        a = dataset[i:(i+timesteps), 0]
        dataX.append(a)
        dataY.append(dataset[i + timesteps, 0])
    return numpy.array(dataX), numpy.array(dataY)

# fix random seed for reproducibility 
numpy.random.seed() 

# load the dataset
t = '00_Train_2003.11.25.12.17_14.07_12sets.csv' # 12sets 
dataframe = pandas.read_csv(t, header=None, sep='	', usecols=[4], engine='python')  
dataset = dataframe.values  # 여기는 항상 t 시점
dataset = dataset.astype('float32')
print(' dataset.shape == ', dataset.shape)

t1 = '00_Test_2003.11.25.14.17_16.07_12sets.csv' # 12sets 
dataframe2 = pandas.read_csv(t1, header=None, sep='	', usecols=[4], engine='python')
dataset2 = dataframe2.values  # Prediction Target(label) Data
dataset2 = dataset2.astype('float32')
print(' dataset2.shape == ', dataset2.shape)

# normalize the dataset ---------------- # 18.2.25. 
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

scaler2 = MinMaxScaler(feature_range=(0, 1)) # New 18.3.16.금.14시34분
dataset2 = scaler2.fit_transform(dataset2)

print(' dataset.shape == ', dataset.shape)
print(' dataset2.shape == ', dataset2.shape)

## normalize the dataset ---------------# New 18.5.21.월.15시30분
mean1 = numpy.mean(dataset)  
print('mean of train == ', mean1)
print('     mean type of train -- ' , type(numpy.mean(dataset)) )
# dataset -= mean1 ### 차분, 노멀라이즈 
## 분산 std1 = sum((dataset-numpy.mean(dataset)**2))/int(len(dataset))  
std1 = numpy.std(dataset) # 표준편차 std 
print('std of train == ', std1)
# dataset /= std1 ### 차분, 노멀라이즈 
## normalize the dataset ---------------# New 18.5.21.월.15시30분
mean2 = numpy.mean(dataset2)  
print('mean of test == ', mean2)
print('     mean type of test -- ' , type(numpy.mean(dataset2)) )
# dataset2 -= mean2 ### 차분, 노멀라이즈 
## 분산 std1 = sum((dataset-numpy.mean(dataset)**2))/int(len(dataset))  
std2 = numpy.std(dataset2) # 표준편차 std 
print('std of test == ', std2)
# dataset2 /= std2 ### 차분, 노멀라이즈

# split into train and test sets ----------------
train_size = int(len(dataset)) # train 102400/5(batch_size)=  ,102396 ->102380   
test_size = int(len(dataset2)) # test 20480/5(batch_size)=   ,20478
train = dataset[0:train_size,:] 
test = dataset2[0:test_size,:]
print('length of train == ', int(len(dataset)))
print('length of test == ', int(len(dataset2))) 
print('length of Total == ', int(len(dataset))+int(len(dataset2))) 
print(' Lets start ~!!! go, go! ') 

# reshape into X=t and Y=t+1
features = 1
timesteps = 32 # 29 # batch_size(3) X 3, 100설정 시, epoch 당 8시간 소요, timesteps=lookback은 한 스텝을 예측 

trainX, trainY = create_dataset(train, timesteps)
testX, testY = create_dataset(test, timesteps)
# reshape input to be [samples, time steps, features] 타임스텝이란 하나의 샘플에 포함된 시퀀스 갯수
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
print('-----------------------------------')  
print(' train_X.shape = ', trainX.shape) # train_X.shape = (204770, 29, 1)
print(' train_Y.shape = ', trainY.shape) # train_Y.shape = (204770,)
print(' test_X.shape == ', testX.shape)  # test_X.shape == (40930, 29, 1) 
print(' test_Y.shape == ', testY.shape)  # test_Y.shape == (40930,)
print('-----------------------------------')  

# create and fit the CNN and LSTM network
# 2018.12.9.일.23시.
# visible1 = Input(shape=(204770, 29))  #### 아래 strides를 1로 해야 acc향상 
# visible1 = Input(shape=(timesteps, features))  # 29, 1
# filter = output, must same, kernel = input 
#conv11 = Conv1D(filters=32, kernel_size=1, strides=1, activation='relu')(visible1)
####### BN 하지 않는다. Pooling 하지 않는다. ####### 
####### 1D에서는 pooling을 하면 성능 악화. 1D에서는 BN도 성능 악화.  


# C(32)-C(32)-BL(4)-BL(8)-L(16)-L(16)
# Dropout 0.5 

######################################### with K.tf.device('/gpu:0'):  => GPU usage
import keras.backend.tensorflow_backend as K 
with K.tf.device('/gpu:0'):
    visible1 = Input(shape=(timesteps, features))  # 32, 1 # 29, 1
     
    conv_11 = Conv1D(filters=64, kernel_size=1, strides=1, activation='relu')(visible1)
    conv_11_out = Dropout(0.5)(conv_11)
    conv_12 = Conv1D(filters=32, kernel_size=1, strides=1, activation='relu')(conv_11_out)
    conv_12_out = Dropout(0.5)(conv_12)
    
    #residual_1 = keras.layers.Add()([visible1, conv_12_out]) # 32, 32
    #bi11 = (Bidirectional(LSTM(8, return_sequences=True, dropout=0.5)))(residual_1) ### success 
    
    bi11 = (Bidirectional(LSTM(8, return_sequences=True, dropout=0.5)))(conv_12_out) ### success 
    bi12 = (Bidirectional(LSTM(16, return_sequences=True, dropout=0.5)))(bi11)  
    #bi11 = (Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(0.001))))(residual_1) ### success 
    #bi12 = (Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(0.001))))(bi11)  
    
    #residual_2 = keras.layers.Add()([residual_1, bi12]) # 32, 32 (16x2)
    #ls11 = (LSTM(32, return_sequences=True, dropout=0.5))(residual_2) # 32
    
    ls11 = (LSTM(32, return_sequences=True, dropout=0.5))(bi12) # 32
    ls12 = (LSTM(32, dropout=0.5))(ls11) # do not use retrun_sequence=True at last lstm layer 
    #ls11 = (LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001)))(residual_2) # 32
    #ls12 = (LSTM(16, kernel_regularizer=l2(0.001)))(ls11) # do not use retrun_sequence=True at last lstm layer 
    
    ##########  (29, 16) (12,) ,  (29, 16) (29,)
    # res12 = keras.layers.Add()([res11, ls12]) #########
    dense11 = Dense(8, activation='relu')(ls12)
    out1 = Dense(1, activation='relu')(dense11)
    model = Model(inputs=[visible1], outputs=[out1]) # multi-input, multi-output 
    
    # from keras.regularizers import l2
    # keras.regularizers.l1(0.)
    # keras.regularizers.l2(0.)
    # keras.regularizers.l1_l2(l1=0.01, l2=0.01)

    # = (LSTM(32, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))
    # = Conv2D(32, (3,3), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    # = Dense(32, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    # = Conv1D(filters=128, kernel_size=1, strides=1, kernel_regularizer=regularizers.l2(0.001))(conv_31)             
    #########################################
    optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False) 

    #checkpoint = ModelCheckpoint(saveweightfile, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='auto', period=1)

    reducelr = ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=1) #에포크 1동안 좋아지지 않으면 호출 
    
    csv_logger = CSVLogger(title + 'log.csv', append=True, separator='\n')  # separator=';', separator='\t'
        
    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  #optimizer='adam',
                  metrics=['mse', 'mae', 'mape'])
    
    batch_size = batch_size  #  avove gloval varibale 
    epochs=epochs   # avove gloval varibale  
    
    history = LossHistory()#callbacks=[history] above model.fit
    history.init()         #callbacks=[history] above model.fit
    
    for i in range(epochs):   ### 10 or 20, 아래 epochs가 1,일, 따라서 1 X 수정10 = 10회 epochs
        model.fit(trainX, trainY, 
                  epochs=1, 
                  batch_size=batch_size, 
                  shuffle=False,   
                  validation_data=(testX, testY),
                  callbacks=[history, reducelr, csv_logger])  # , checkpoint  
        model.reset_states() # shuffle=False 순서대로 출력의미, LSTM레이어가 여럿일때는 fit(return_sequence=True) 
#################################################
# make predictions
print(' Start Prediction ... ')   
print(' Train X_1 (bearing 3 x)  Predicting...') 
trainPredict = model.predict(trainX, batch_size=batch_size, verbose = 1)
model.reset_states()
print(' Test X_1 (bearing 3 x) Predicting...') 
testPredict = model.predict(testX, batch_size=batch_size, verbose = 1)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict) #scaler2 
testY = scaler.inverse_transform([testY]) #scaler2 

# calculate root mean squared error 
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.5f RMSE' % (trainScore))  ## 1-RMSE = Accuracy %  
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.5f RMSE' % (testScore))  ## 1-RMSE = Accuracy %  
print('--------------------------------------------------------') 
print(' history.losses      = ', history.losses) # loss == mse 
print(' history.mses(=loss) = ', history.mses)  # loss == mse 
print(' history.maes        = ', history.maes)
print(' history.mapes       = ', history.mapes)
print(' history.val_losses     = ', history.val_losses)  # loss == mse 
print(' history.val_mses(=loss)= ', history.val_mses) # loss == mse 
print(' history.val_maes       = ', history.val_maes)
print(' history.val_mapes      = ', history.val_mapes)
print('--------------------------------------------------------')      
# ---------------------------------
# print to # Layer, Shape, Parameter 
print('model summary ... ', model.summary())  # keras.models.model()
# ---------------------------------
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[timesteps:len(trainPredict)+timesteps, :] = trainPredict
# shift test predictions for plotting ... NEW
testPredictPlot = numpy.empty_like(dataset2)
testPredictPlot[:, :] = numpy.nan
#testPredictPlot[len(trainPredict)+(timesteps*2)+1:len(dataset2)-1, :] = testPredict
testPredictPlot[timesteps:len(testPredict)+timesteps, :] = testPredict
# ---------------------------------
plt.plot(scaler.inverse_transform(dataset), label='Groundtruth')
plt.plot(trainPredictPlot, label='Training')
plt.title('Train of Bearing3 X-axis from 12:17 to 14:07') 
plt.ylabel('Frequency')
plt.xlabel('Samples')
plt.legend() ## plt.plot(,label='')
plt.savefig(title+'_Train_groundtruth.png') ###### 수정 
plt.show()
# ---------------------------------
plt.plot(scaler2.inverse_transform(dataset2), label='Groundtruth')
#plt.plot(trainPredictPlot) 불필요, 아래 test display
plt.plot(testPredictPlot, label='Test')
plt.title('Prediction Test of Bearing3 X-axis from 14:17 to 16:07') 
plt.ylabel('Frequency')
plt.xlabel('Samples')
plt.legend() ## plt.plot(,label='')
plt.savefig(title+'_Test_groundtruth.png')  ###### 수정 
plt.show()
print('--------------------------------------------------------')  
print(' history.losses(mse)     == ', history.losses) 
print(' history.val_losses(mse) == ', history.val_losses)
print('--------------------------------------------------------') 
plt.plot(history.mses, 'b--', label='MSE of Train') # Train 0.000x소수점단위
plt.plot(history.maes, 'b:', label='MAE of Train') # Train 0.000x소수점단위
plt.plot(history.val_mses, 'ro', label='MSE of Test') # Test 0.000x 소수점단위
plt.plot(history.val_maes, 'r+', label='MAE of Test') # Test 0.000x 소수점단위 
plt.title('Train loss and Test loss of Bearing3 X-axis') 
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend() ## plt.plot(,label='')
plt.savefig(title+'_mse_mae.png') ###### 수정 
plt.show()
print('--------------------------------------------------------')
print('----------------------- The end of code ----------------') 
"""
plt.plot(history.losses, 'bo')  # Train losses = MSE 
plt.plot(history.val_losses, 'b') # Test losses = MSE 
##### 아래 주의, x축은 epochs ### 
plt.axis([0, epochs, 0.00, 0.04]) ### y-end의 val_loss가 0.22를 넘지 않음.
##### x-start(0), x-end(에폭 수), y-start(0), y-and #####
plt.title('Model Loss, MAE of Bearing3 X-axis from 14:17 to 16:07')
plt.ylabel('Loss(Mean Squared Error)')
plt.xlabel('Epoch')
#plt.legend(['Train', 'Test'], loc=0)  #plt.plot(,,,label=)
plt.legend(['Train', 'Test'])
plt.savefig(title+'_MAE_Model_Loss.png')
#plt.figure() ## 
plt.show() 
print('--------------------------------------------------------')  
plt.plot(history.mapes, 'bo') # Train MAPE
plt.plot(history.val_mapes, 'b') # Test MAPE
##### 아래 주의, x축은 epochs ### 
plt.axis([0, epochs, 0.00, 60000]) ### y-end의 val_loss가 30000를 넘지 않음.
##### x-start(0), x-end(에폭 수), y-start(0), y-and #####
plt.title('Prediction Loss(MAPE) of Bearing3 X-axis from 14:17 to 16:07')
plt.ylabel('Loss(Mean Absolute Percentage Error)')
plt.xlabel('Epoch')
#plt.legend(['Train', 'Test'], loc=0)  #plt.plot(,,,label=)
plt.legend(['Train', 'Test'])
plt.savefig(title+'_MAPE_Predict_Loss.png')
#plt.figure() ## 
plt.show() 
print('--------------------------------------------------------')  
plt.plot(history.mses, label='MSE of Train') # Train 0.000x소수점단위
plt.plot(history.maes, label='MAE of Train') # Train 0.000x소수점단위
#plt.plot(history.mapes, label='mean_absolute_percentage_erro') # 1000~2000단위, 출력만 제외 
##### 아래 주의, x축은 epochs ### 
plt.axis([0, epochs, 0.000, 0.040]) ### y-end의 val_loss가 0.025를 넘지 않음.
##### x-start(0), x-end(에폭 수), y-start(0), y-and #####
plt.title('Train Loss of Bearing3 X-axis from 12:17 to 14:07') 
plt.ylabel('Train Loss')
plt.xlabel('Train Epoch')
plt.legend() ## plt.plot(,label='')
plt.savefig(title+'_Train_Loss.png') ###### 수정 
plt.show()
print('--------------------------------------------------------') 
plt.plot(history.val_mses, label='Test_MSE') # Test 0.000x 소수점단위
plt.plot(history.val_maes, label='Test_MAE') # Test 0.000x 소수점단위 
#plt.plot(history.val_mapes, label='validation_MAPE') # 1000~2000단위, 출력만 제외 
##### 아래 주의, x축은 epochs ### 
plt.axis([0, epochs, 0.00, 0.28]) ### y-end의 val_loss가 0.14를 넘지 않음.
##### x-start(0), x-end(에폭 수), y-start(0), y-and #####
plt.title('Prediction Test Loss of Bearing3 X-axis from 14:17 to 16:07') 
plt.ylabel('Test Loss')
plt.xlabel('Test Epoch')
plt.legend() ## plt.plot(,label='')
plt.savefig(title+'_Test_Loss.png') ###### 수정 
plt.show()
print('--------------------------------------------------------') 
# ---------------------------------
series1 = dataframe #dataframe = pandas.read_csv(t, header=None, sep='	', usecols=[4], engine='python')
series1.plot(kind='kde') # ‘kde’ : Kernel Density Estimation plot, ‘line’ : line plot (default) 
plt.title('Density Plots of Train Dataset(Bearing3 X-axis from 12:17 to 14:07)') 
plt.savefig(title+'_dataset_DensityPlots.png') 
plt.show()
# ---------------------------------
series2 = dataframe2 #dataframe2 = pandas.read_csv(t1, header=None, sep='	', usecols=[4], engine='python')
series2.plot(kind='kde') # ‘kde’ : Kernel Density Estimation plot, ‘line’ : line plot (default) 
plt.title('Density Plots of Test Dataset(Bearing3 X-axis from 14:17 to 16:07)') 
plt.savefig(title+'_dataset2_DensityPlots.png') 
plt.show()
# ---------------------------------
# box and whisker plot
# from pandas import DataFrame
df_dataset = pandas.DataFrame(dataset) #Train dataset
df_dataset.boxplot() 
plt.savefig(title+'_boxplot_dataset.png') 
plt.show() 
print('------- df_dataset.describe() -------')
print(df_dataset.describe())
# -------------------------------------
df_trainPredictPlot = pandas.DataFrame(trainPredictPlot)
df_trainPredictPlot.boxplot()  
plt.savefig(title+'_boxplot_trainpredict.png') 
plt.show() 
print('------- df_trainPredictPlot.describe() -------')
print(df_trainPredictPlot.describe())
# -------------------------------------
# box and whisker plot
# from pandas import DataFrame
df_dataset2 = pandas.DataFrame(dataset2) #Test dataset
df_dataset2.boxplot() 
plt.savefig(title+'_boxplot_dataset2.png') 
plt.show() 
print('------- df_dataset2.describe() -------')
print(df_dataset2.describe())
# -------------------------------------
df_testPredictPlot = pandas.DataFrame(testPredictPlot)
df_testPredictPlot.boxplot()  # aa.boxplot()에서 aa는 dataframe 타입 
plt.savefig(title+'_boxplot_test_predict.png') 
plt.show() 
print('------- df_testPredictPlot.describe() -------')
print(df_testPredictPlot.describe())
# -----------------------------------------------------
"""
# end of code

Using TensorFlow backend.


 dataset.shape ==  (245760, 1)
 dataset2.shape ==  (245760, 1)
 dataset.shape ==  (245760, 1)
 dataset2.shape ==  (245760, 1)
mean of train ==  0.45230213
     mean type of train --  <class 'numpy.float32'>
std of train ==  0.032692585
mean of test ==  0.5687847
     mean type of test --  <class 'numpy.float32'>
std of test ==  0.033779364
length of train ==  245760
length of test ==  245760
length of Total ==  491520
 Lets start ~!!! go, go! 
-----------------------------------
 train_X.shape =  (245727, 32, 1)
 train_Y.shape =  (245727,)
 test_X.shape ==  (245727, 32, 1)
 test_Y.shape ==  (245727,)
-----------------------------------
Train on 245727 samples, validate on 245727 samples
Epoch 1/1
245727/245727 [==============================] - 550s 2ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - mean_absolute_error: 0.0509 - mean_absolute_percentage_error: 1848.2920 - val_loss: 0.0140 - val_mean_squared_error: 0.0140 - val_mean_absolute_error: 0.1135 - val_mean_absolute_percent

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

--------------------------------------------------------
 history.losses(mse)     ==  [0.008123002631343326, 0.0010917002549883088, 0.0010780426455392058, 0.0010748781570731106, 0.0010730917976060242, 0.0010721202837301273, 0.0010719150928421935, 0.001071552364249246, 0.0010714097091411886, 0.0010714132143226879]
 history.val_losses(mse) ==  [0.013977169857777182, 0.013945639291017957, 0.013994897166365313, 0.014032326407462607, 0.014115573710150243, 0.014186185192447043, 0.01420828700834618, 0.014218754735172994, 0.014226676411199204, 0.01422208130222434]
--------------------------------------------------------


<Figure size 640x480 with 1 Axes>

--------------------------------------------------------
----------------------- The end of code ----------------


"\nplt.plot(history.losses, 'bo')  # Train losses = MSE \nplt.plot(history.val_losses, 'b') # Test losses = MSE \n##### 아래 주의, x축은 epochs ### \nplt.axis([0, epochs, 0.00, 0.04]) ### y-end의 val_loss가 0.22를 넘지 않음.\n##### x-start(0), x-end(에폭 수), y-start(0), y-and #####\nplt.title('Model Loss, MAE of Bearing3 X-axis from 14:17 to 16:07')\nplt.ylabel('Loss(Mean Squared Error)')\nplt.xlabel('Epoch')\n#plt.legend(['Train', 'Test'], loc=0)  #plt.plot(,,,label=)\nplt.legend(['Train', 'Test'])\nplt.savefig(title+'_MAE_Model_Loss.png')\n#plt.figure() ## \nplt.show() \nprint('--------------------------------------------------------')  \nplt.plot(history.mapes, 'bo') # Train MAPE\nplt.plot(history.val_mapes, 'b') # Test MAPE\n##### 아래 주의, x축은 epochs ### \nplt.axis([0, epochs, 0.00, 60000]) ### y-end의 val_loss가 30000를 넘지 않음.\n##### x-start(0), x-end(에폭 수), y-start(0), y-and #####\nplt.title('Prediction Loss(MAPE) of Bearing3 X-axis from 14:17 to 16:07')\nplt.ylabel('Loss(Mean Absolute Percentage Er